In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import plotly.express as px
import plotly.graph_objs as go

import torch
import torch.nn as nn

In [ ]:
lm.LinearRegression?

In [ ]:
gender = pd.read_csv("gender.csv")
gender = gender.drop(['Unnamed: 0', 'School'], axis = 1)

eth = pd.read_csv("race.csv")
eth = eth.drop(['Unnamed: 0', 'School'], axis = 1).rename(columns = {'Uad Uc Ethn 6 Cat': 'Ethnicity'})

In [ ]:
gender

In [ ]:
print(gender[['Calculation1', 'County/State/ Territory', 'Count', 'Gender', 'Pivot Field Values']] \
.rename(columns = {'Pivot Field Values': 'Value', 
                   'County/State/ Territory': 'Location',
                   'Calculation1': 'Name'}) \
.sample(5, random_state=10060000) \
.to_latex(index=False))

In [ ]:
print(gender[['Calculation1', 'County/State/ Territory', 'Count', 'Gender', 'Pivot Field Values']] \
.rename(columns = {'Pivot Field Values': 'Value', 
                   'County/State/ Territory': 'Location',
                   'Calculation1': 'Name'}) \
.iloc[11:19, :] \
.to_latex(index=False))

In [ ]:
print(gender.head().to_latex())

In [ ]:
print(eth.head().to_latex())

In [ ]:
# features: proportion of applicants that were male
# proportion admitted male

# target: in state or not AND in state, out of state, international

In [ ]:
# we need schools with both numbers for # of students applied and # of students admitted
# so we need to do some wranging to get that

In [ ]:
gender_filtered = gender.groupby('Calculation1').filter(lambda df: len(df.loc[df['Count'] == 'Adm', 'Count']) > 0)

In [ ]:
gender_filtered

In [ ]:
# not all schools have numbers for all categories – we need to interpolate them for consistency

In [ ]:
all_count, male_count, female_count = 0, 0, 0

In [ ]:
for name, df in gender_filtered.groupby('Calculation1'):
    app = df.loc[df['Count'] == 'App', 'Gender']
    app = sorted(app.tolist())
    
    if app == ['All']:
        # No gender data
        pass
    
    if app == sorted(['All', 'Female', 'Male']):
#         print('has all')
        all_count += 1

    if app == ['All', 'Male']:
#         print(name)
        male_count += 1
    
    if app == ['All', 'Female']:
        female_count += 1
    
#     print(df)

In [ ]:
# jk – the majority of schools do have all of that data, let's just work with them

In [ ]:
gender_filtered = gender_filtered.groupby('Calculation1').filter(
    lambda df: (sorted(df.loc[df['Count'] == 'App', 'Gender'].tolist()) == ['All', 'Female', 'Male']) &
               (sorted(df.loc[df['Count'] == 'Adm', 'Gender'].tolist()) == ['All', 'Female', 'Male'])

)

In [ ]:
gender_filtered

In [ ]:
df = pd.pivot_table(gender_filtered,
               index = 'Calculation1', 
               values = 'Pivot Field Values', 
               columns = ['Count', 'Gender'])[['App', 'Adm']]

In [ ]:
df

In [ ]:
df['AppMaleProportion'] = df[('App', 'Male')] / df[('App', 'All')]
df['AdmMaleProportion'] = df[('Adm', 'Male')] / df[('Adm', 'All')]

In [ ]:
# https://stackoverflow.com/questions/14507794/pandas-how-to-flatten-a-hierarchical-index-in-columns
df.columns = [''.join(col).strip() for col in df.columns.values]

In [ ]:
df

In [ ]:
# getting whether or not the school is in state

In [ ]:
# international
gender_filtered.loc[gender_filtered['County/State/ Territory'].isnull(), 'Location'] = 'INT'
gender_filtered.loc[gender_filtered['County/State/ Territory'].str.len() == 2, 'Location'] = 'OOS'
gender_filtered.loc[gender_filtered['County/State/ Territory'].str.len() > 2, 'Location'] = 'INS'

In [ ]:
df = df.merge(gender_filtered[['Calculation1', 'Location']].groupby('Calculation1').agg(lambda x: x.iloc[0])
         , on = 'Calculation1')

In [ ]:
df

In [ ]:
# now we need to repeat the above filtering for ethnicity, join the two, and then start classifying stuff
# for simplicity, we just need the 'white' proportion

In [ ]:
eth

In [ ]:
eth_filtered = eth.groupby('Calculation1').filter(lambda df: len(df.loc[df['Count'] == 'Adm', 'Count']) > 0)
eth_filtered = eth_filtered.groupby('Calculation1').filter(
    lambda df: ('White' in df.loc[df['Count'] == 'App', 'Ethnicity'].tolist()) &
               ('All' in df.loc[df['Count'] == 'App', 'Ethnicity'].tolist()) &
               ('White' in df.loc[df['Count'] == 'Adm', 'Ethnicity'].tolist()) &
               ('All' in df.loc[df['Count'] == 'Adm', 'Ethnicity'].tolist())
)

eth_filtered.loc[eth_filtered['County/State/ Territory'].isnull(), 'Location'] = 'INT'
eth_filtered.loc[eth_filtered['County/State/ Territory'].str.len() == 2, 'Location'] = 'OOS'
eth_filtered.loc[eth_filtered['County/State/ Territory'].str.len() > 2, 'Location'] = 'INS'

df_eth = pd.pivot_table(eth_filtered,
               index = 'Calculation1', 
               values = 'Pivot Field Values', 
               columns = ['Count', 'Ethnicity'])[['App', 'Adm']]

df_eth['AppWhiteProportion'] = df_eth[('App', 'White')] / df_eth[('App', 'All')]
df_eth['AdmWhiteProportion'] = df_eth[('Adm', 'White')] / df_eth[('Adm', 'All')]

df_eth.columns = [''.join(col).strip() for col in df_eth.columns.values]

df_eth = df_eth[['AppWhiteProportion', 'AdmWhiteProportion']]

df_eth = df_eth.merge(eth_filtered[['Calculation1', 'Location']].groupby('Calculation1').agg(lambda x: x.iloc[0])
         , on = 'Calculation1')

In [ ]:
# turns out no schools both reported white/all proportions AND male/female/all proportions, so these analyses need to be done separately

In [ ]:
df

### actual machine learning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df.head()

In [ ]:
df['binary target'] = 1 * (df['Location'] == 'INS')

In [ ]:
df['Location'].value_counts()

In [ ]:
df['binary target'].value_counts()

In [ ]:
train_binary, test_binary = train_test_split(df, test_size = 0.2, random_state = 42)

In [ ]:
binary_models = {
    'lr': LogisticRegression(),
    'lrb': LogisticRegression(class_weight = "balanced"),
    'dt': DecisionTreeClassifier(),
    'rf': RandomForestClassifier()
}

In [ ]:
for m in binary_models:
    binary_models[m].fit(train_binary[['AppMaleProportion', 'AdmMaleProportion']], train_binary['binary target'])
    train_acc = binary_models[m].score(train_binary[['AppMaleProportion', 'AdmMaleProportion']], train_binary['binary target'])
    test_acc = binary_models[m].score(test_binary[['AppMaleProportion', 'AdmMaleProportion']], test_binary['binary target'])
    print(m, train_acc, test_acc)
    
    sns_cmap = ListedColormap(np.array(sns.color_palette())[0:2, :])

    xx, yy = np.meshgrid(np.arange(0, 1, 0.01), np.arange(0, 1, 0.01))
    Z_string = binary_models[m].predict(np.c_[xx.ravel(), yy.ravel()])
    categories, Z_int = np.unique(Z_string, return_inverse = True)
    Z_int = Z_int.reshape(xx.shape)
    print(categories)
    plt.contourf(xx, yy, Z_int, cmap = sns_cmap)
    sns.scatterplot(data = train_binary, x = 'AppMaleProportion', y = 'AdmMaleProportion', hue = 'binary target', cmap = sns_cmap)
#     plt.title('Logistic Regression on nba_train');
#     break
#     plt.show()
    

In [ ]:
binary_models['dt'].predict(train_binary[['AppMaleProportion', 'AdmMaleProportion']])

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
# lr cm
confusion_matrix(test_binary['binary target'], binary_models['lr'].predict(test_binary[['AppMaleProportion', 'AdmMaleProportion']])) / len(test_binary)

In [ ]:
train_binary['binary target'].value_counts()

In [ ]:
len(train_binary)

In [ ]:
confusion_matrix?

In [ ]:
# lrb cm
confusion_matrix(test_binary['binary target'], binary_models['lrb'].predict(test_binary[['AppMaleProportion', 'AdmMaleProportion']])) / len(test_binary)

In [ ]:
train_binary['binary target'].value_counts()

In [ ]:
test_binary['binary target'].value_counts()

In [ ]:
# merged data

In [ ]:
df_merged = df.merge(df_eth, left_index = True, right_index = True)[['AppMaleProportion', 'AdmMaleProportion',
                                                       'AppWhiteProportion', 'AdmWhiteProportion',
                                                       'Location_x', 'binary target']].rename(
                                                        columns = {'Location_x': 'Location'})
df_merged

In [ ]:
df_export = df_merged.copy()

In [ ]:
df_export.index.name = 'Name'
df_export = df_export.iloc[:, :-1]
df_export.index = df_export.index.to_series().str[:10]
df_export.columns = ['AppMale', 'AdmMale', 'AppWhite', 'AdmWhite', 'Location']

In [ ]:
df_export.sample(5, random_state=53)

In [ ]:
print(df_export.sample(5, random_state=53).head().to_latex())

In [ ]:
df_merged.corr()

In [ ]:
train_merged, test_merged = train_test_split(df_merged, test_size = 0.2, random_state = 6)

In [ ]:
binary_models_merged = {
    'lr': LogisticRegression(),
    'lrb': LogisticRegression(class_weight = "balanced"),
    'dt': DecisionTreeClassifier(),
    'rf': RandomForestClassifier()
}

In [ ]:
features = ['AppMaleProportion', 'AdmMaleProportion', 'AppWhiteProportion', 'AdmWhiteProportion']

In [ ]:
for m in binary_models_merged:
    binary_models_merged[m].fit(train_merged[features], train_merged['binary target'])
    train_acc = binary_models_merged[m].score(train_merged[features], train_merged['binary target'])
    test_acc = binary_models_merged[m].score(test_merged[features], test_merged['binary target'])
    print(m, train_acc, test_acc)
    
    # could add in precision and recall
    

In [ ]:
# roc curve of binary logistic regression model

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
fpr, tpr, threshold = roc_curve(train_merged['binary target'], 
                               binary_models_merged['lr'].predict_proba(train_merged[features])[:, 1])

plt.plot(fpr, tpr);
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC, Unbalanced Four-Feature Log. Reg.');
print('auc = ', auc(fpr, tpr))
# plt.savefig('export/unbalancedroc.png')

In [ ]:
fpr, tpr, threshold = roc_curve(train_merged['binary target'], 
                               binary_models_merged['lrb'].predict_proba(train_merged[features])[:, 1])

plt.plot(fpr, tpr);
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC, Balanced Four-Feature Log. Reg.');
print('auc = ', auc(fpr, tpr))
# plt.savefig('export/balancedroc.png')

In [ ]:
binary_models['lrb'].coef_, binary_models['lrb'].intercept_, 

In [ ]:
binary_models_merged['lrb'].coef_, binary_models_merged['lrb'].intercept_

In [ ]:
# lrb cm
confusion_matrix(test_merged['binary target'], binary_models_merged['lr'].predict(test_merged[['AppMaleProportion', 'AdmMaleProportion', 'AppWhiteProportion', 'AdmWhiteProportion']]))

In [ ]:
# lrb cm
confusion_matrix(test_merged['binary target'], binary_models_merged['lrb'].predict(test_merged[['AppMaleProportion', 'AdmMaleProportion', 'AppWhiteProportion', 'AdmWhiteProportion']]))

### fairness criterion

In [ ]:
Y = train_merged['binary target']
A = train_merged[features]
R = binary_models_merged['lrb'].predict(train_merged[features])#[:, 1]

In [ ]:
# clearly these are garbage, pretty close to random model (0.5)
# only classifying well because of the severe class imbalance

In [ ]:
# plan: make A the category for whether or not a majority of applicants were male (or accepted)
# then look at separation and sufficiency

In [ ]:
A = train_merged['AppMaleProportion'] >= 0.5

In [ ]:
plt.scatter(R[A], Y[A], label = 'male majority', alpha = 0.4, color = 'r');
# plt.scatter(R[~A], Y[~A], label = 'female majority', alpha = 0.4);
plt.legend();

In [ ]:
plt.figure(figsize=(5,5))
plt.errorbar([0], [np.mean(Y[A][R[A] == 0])], color = 'red', alpha = 0.4, label = 'male',
            yerr=np.std(Y[A][R[A] == 0]) / 15, fmt='o') 
plt.errorbar([1], [np.mean(Y[A][R[A] == 1])], color = 'red', alpha = 0.4,
            yerr=np.std(Y[A][R[A] == 1]) / 15, fmt='o')

plt.errorbar([0], [np.mean(Y[~A][R[~A] == 0])], color = 'blue', alpha = 0.4, label = 'female',
            yerr=np.std(Y[~A][R[~A] == 0]) / 15, fmt='o')
plt.errorbar([1], [np.mean(Y[~A][R[~A] == 1])], color = 'blue', alpha = 0.4,
            yerr=np.std(Y[~A][R[~A] == 1]) / 15, fmt='o');
plt.legend()
plt.ylim([0.75,1])
plt.title("Sufficiency")
plt.ylabel("P(Y=y | R=r, A=a)")
plt.xlabel("R")
plt.show()

In [ ]:
offset = 0.06
w = 0.1
plt.figure(figsize=(8,5))
l = len(R)
# average predicted oos given male
plt.bar([0-offset], np.mean(R[A][Y[A] == 0]), color='red', label='male', alpha = 0.4,width=w)
# average predicted oos given female
plt.bar([0+offset], np.mean(R[~A][Y[~A] == 0]), color='blue', label='female', alpha = 0.4,width=w)

# average predicted instate given male
plt.bar([1-offset], np.mean(R[A][Y[A] == 1]), color='red', alpha = 0.4,width=w)
# average predicted instate given female
plt.bar([1+offset], np.mean(R[~A][Y[~A] == 1]), color='blue', alpha = 0.4,width=w)
plt.legend()
plt.title("Separation")
plt.ylabel("P(R=r | Y=y, A=a)")
plt.xlabel("Y")
plt.show()

In [ ]:
print(len(train_merged['AppMaleProportion']))

In [ ]:
print(sum(~A))

In [ ]:
np.mean(Y[~A][R[~A] == 1])

In [ ]:
np.mean(Y[~A][R[~A] == 0])

In [ ]:
# out of state schools have more female applicants??? 

## Regression models

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
# given all other features, predict AdmWhiteProportion

In [ ]:
df_ohe = pd.get_dummies(df_merged, columns = ['Location']) \
.drop(['binary target', 'Location_INS'], axis = 1) \
.rename(columns = {'AdmWhiteProportion': 'TargetProportion'}) \
.iloc[:, [0, 1, 2, 4, 5, 3]]

In [ ]:
df_ohe

In [ ]:
train_ohe, test_ohe = train_test_split(df_ohe, test_size = 0.2, random_state = 17)

In [ ]:
regression_models = {
    'ols': LinearRegression(),
    'ridge': Ridge(),
    'ridgecv': RidgeCV(),
    'lasso': Lasso(),
    'lassocv': LassoCV(),
    'knn': KNeighborsRegressor()
}

In [ ]:
for m in regression_models:
    regression_models[m].fit(train_ohe.iloc[:, :-1], train_ohe.iloc[:, -1])
    train_rmse = np.mean((regression_models[m].predict(train_ohe.iloc[:, :-1]) - train_ohe.iloc[:, -1])**2)
    test_rmse = np.mean((regression_models[m].predict(test_ohe.iloc[:, :-1]) - test_ohe.iloc[:, -1])**2)
    if m == 'knn':
        print(m, train_rmse, test_rmse)
    else:
        print(f"{m}, {np.round(train_rmse, 3)}, {np.round(test_rmse, 3)}, {np.round(regression_models[m].coef_, 3)}, {np.round(regression_models[m].intercept_, 3)}")
    

In [ ]:
train_ohe.iloc[:, -1].mean()

In [ ]:
regression_models['lasso'].predict(train_ohe.iloc[:, :-1])

In [ ]:
df_ohe

In [ ]:
train_merged

## PyTorch Code

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [ ]:
class NNClassifier(nn.Module):
    def __init__(self, in_feat, hidden_dims=(1000, 500), extra_layer=False):
        super().__init__()
        self.extra_layer = extra_layer
        self.fc1 = nn.Linear(in_feat, hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.fc3 = nn.Linear(hidden_dims[1], hidden_dims[1])
        self.fc4 = nn.Linear(hidden_dims[1], 2)
        
    def forward(self, x):
        l1_out = nn.functional.relu(self.fc1(x))
        l2_out = nn.functional.relu(self.fc2(l1_out))
        l3_out = nn.functional.relu(self.fc3(l2_out)) if self.extra_layer else l2_out
        out = self.fc4(l3_out)
        return out
    
class NNRegressor(nn.Module):
    def __init__(self, in_feat, hidden_dims=(50, 25), extra_layer=False):
        super().__init__()
        self.extra_layer = extra_layer
        self.fc1 = nn.Linear(in_feat, hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0], hidden_dims[1])
        self.fc3 = nn.Linear(hidden_dims[1], hidden_dims[1])
        self.fc4 = nn.Linear(hidden_dims[1], 1)
        
    def forward(self, x):
        l1_out = nn.functional.relu(self.fc1(x))
        l2_out = nn.functional.relu(self.fc2(l1_out))
        l3_out = nn.functional.relu(self.fc3(l2_out)) if self.extra_layer else l2_out
        out = self.fc4(l3_out)
        return out

In [ ]:
test_ohe.iloc[:, -1].values[:,None].shape

In [ ]:
# Converting the DataFrame to a DataLoader
def df_to_torch(data, target, batch_size=5, classify=True):
    if classify:
        target = torch.tensor(target.values.astype(np.long))
    else:
        target = torch.tensor(target.values[:,None].astype(np.float32))
    data = torch.tensor(data.values.astype(np.float32)) 
    data_tensor = TensorDataset(data, target) 
    data_loader = DataLoader(dataset = data_tensor, batch_size = batch_size, shuffle = True)
    return data_loader

# 2 Feature Classifier Data
classify = True
train_data = train_binary[['AppMaleProportion', 'AdmMaleProportion']]
train_target = train_binary['binary target']
test_data = test_binary[['AppMaleProportion', 'AdmMaleProportion']]
test_target = test_binary['binary target']

# 4 Feature Classifier Data
classify = True
# train_data = train_merged[features]
# train_target = train_merged['binary target']
# test_data = test_merged[features]
# test_target = test_merged['binary target']

# 4 Feature Regression
classify = False
train_data = train_ohe.iloc[:, :-1]
train_target = train_ohe.iloc[:, -1]
test_data = test_ohe.iloc[:, :-1]
test_target = test_ohe.iloc[:, -1]

train_loader = df_to_torch(train_data, train_target, classify=classify)
test_loader = df_to_torch(test_data, test_target, classify=classify)

In [ ]:
num_pos = sum(train_target.values)
total = len(train_target.values)
num_neg = total-num_pos
weights = torch.tensor([num_neg, num_pos]).type(torch.float)/total
weights

In [ ]:
# Create NN
# net = NNClassifier(2, hidden_dims=(50, 25))
# net = NNClassifier(4, hidden_dims=(1000, 500), extra_layer=True)
net = NNRegressor(5)

In [ ]:
net

In [ ]:
# Training
# criterion = nn.CrossEntropyLoss()
# criterion = nn.CrossEntropyLoss(weight=weights)
criterion = nn.MSELoss(reduction="sum")
# optimizer = optim.SGD(net.parameters(), 0.1)
optimizer = optim.Adam(net.parameters(),lr=0.001)
n_epochs = 200

for epoch in range(n_epochs):
    running_loss = 0
    for (data, target) in train_loader:
        optimizer.zero_grad()
        out = net(data)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()    # Does the update
        
        running_loss += loss.item()
    if epoch % 10 == 0:
        print("Epoch {}, Loss: {}".format(epoch+1, running_loss/total))

In [ ]:
# Calculate Training Accuracy
with torch.no_grad():
        total = 0
        correct = 0
        for i, (data, target) in enumerate(train_loader):
            outputs = net(data)
            _, predicted = torch.max(outputs.data, 1)
#             print(predicted)
            total += target.size(0)
            correct += (predicted == target).sum().item()
        train_acc = correct/total*100
print("Training Accuracy: {:.2f}".format(train_acc))

In [ ]:
# Calculate Testing Accuracy
with torch.no_grad():
        total = 0
        correct = 0
        for i, (data, target) in enumerate(test_loader):
            outputs = net(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
        test_acc = correct/total*100
print("Test Accuracy: {:.2f}".format(test_acc))

In [ ]:
# Calculate Regression Test MSE
with torch.no_grad():
    loss = 0
    total = 0
    for i, (data, target) in enumerate(test_loader):
        loss += sum(((net(data)-target)**2))
        total += data.size(0)
    loss /= total
print("Test MSE: {:.4f}".format(loss.data.numpy()[0]))